<a href="https://colab.research.google.com/github/marcelofschiavo/ds-cookbook/blob/main/07_ML_Clustering_(Nao_Supervisionado).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 07. Aprendizado Não Supervisionado (Clustering)

Nos notebooks 05 e 06 (Supervisionados), nós tínhamos um 'gabarito' (a coluna 'y', como 'pediu_demissao' ou 'salario') e o modelo aprendia a prever esse gabarito.

Aqui, no Não Supervisionado, **não temos gabarito**.
Nosso objetivo é dar os dados "bagunçados" ao modelo e pedir: "Encontre grupos naturais aqui".

**Objetivo:** Segmentar os funcionários em "Personas" distintas (grupos) com base em seus comportamentos e atributos (Ex: 'Salário', 'Satisfação', 'Tempo de Empresa'), para que o RH possa criar ações de engajamento direcionadas.

**Tipo de Problema:** Clustering (Agrupamento).

In [ ]:
"""
## Setup: Carregar Bibliotecas e Dados

Nesta célula, importamos as bibliotecas que usaremos (Pandas e Scikit-learn).
O 'KMeans' e 'silhouette_score' são novos aqui.
"""
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score # Métrica de avaliação de cluster
import warnings

# Ignorar avisos futuros (apenas para limpar o output)
warnings.filterwarnings('ignore')

# --- Carregue seus dados aqui ---
# (Substitua 'dados_empresa_limpos.csv' pelo nome do seu arquivo)
try:
    df = pd.read_csv('dados_empresa_limpos.csv')
    print("DataFrame 'df' carregado com sucesso!")
    print(f"Total de {df.shape[0]} linhas e {df.shape[1]} colunas.")

except FileNotFoundError:
    print("------------------------------------------------------------------")
    print(">>> ERRO: Arquivo 'dados_empresa_limpos.csv' não encontrado. <<<")
    print("... Criando um DataFrame de EXEMPLO para o resto do notebook não quebrar.")
    print("------------------------------------------------------------------")

    # Criar um 'df' de exemplo para clustering
    data = {
        'salario': np.random.randint(3000, 15000, 500),
        'satisfacao': np.random.rand(500).round(2),
        'tempo_empresa': np.random.randint(1, 10, 500),
        'avaliacao_performance': np.random.rand(500).round(2),
        'departamento': np.random.choice(['TI', 'Vendas', 'RH'], 500),
        'pediu_demissao': np.random.choice([0, 1], 500, p=[0.85, 0.15])
    }
    df = pd.DataFrame(data)
    print("DataFrame de exemplo carregado.")

print("\nAmostra dos dados:")
print(df.head())

## Conceito 1: Preparação (Escolha das Features e Scaling)

🧠 **Intuição:**
"Primeiro, escolhemos os 'ingredientes' das nossas personas. O que define um funcionário? Vamos usar `salario`, `satisfacao` e `tempo_empresa`.
Segundo (e **CRÍTICO**): o K-Means (nosso modelo) funciona medindo *distância*. Para ele, uma variação de R$ 1000 no salário parece *infinitamente* maior que uma variação de 0.1 na satisfação.
Precisamos 'equalizar' (padronizar) todas as colunas, colocando-as na mesma régua, para que o modelo as trate com pesos justos."

🎓 **Definição Técnica:**
1.  **Seleção de Features:** Escolhemos as variáveis quantitativas que farão sentido para a segmentação.
2.  **One-Hot Encoding:** Se quisermos usar `departamento`, precisamos transformá-lo em números (dummies) como antes.
3.  **Padronização (StandardScaler):** O K-Means usa Distância Euclidiana, sendo, portanto, *altamente sensível* à escala das features. A padronização (média 0, desvio 1) é **obrigatória** para que features com escalas maiores (como 'salario') não dominem o algoritmo.

🍳 **Receita:**

In [ ]:
from sklearn.preprocessing import StandardScaler

# 1. Selecionar as features que definem as personas
features_para_cluster = [
    'salario',
    'satisfacao',
    'tempo_empresa',
    'avaliacao_performance'
]
# Vamos adicionar 'departamento' também, para mostrar o encoding
df_cluster = pd.get_dummies(df[features_para_cluster + ['departamento']], drop_first=True, dtype=int)


# 2. Padronizar TODAS as features (StandardScaler)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_cluster)

print("Shape dos dados escalados:", X_scaled.shape)
print("\nAmostra dos dados escalados (primeiras 5 linhas):")
print(X_scaled[:5])

📊 **Resultado:**
"Temos uma matriz 'X_scaled' (um array NumPy) onde todas as colunas têm média ~0 e desvio padrão 1, prontas para o K-Means."

## Conceito 2: Encontrando o 'K' (Elbow Method / Método do Cotovelo)

🧠 **Intuição:**
"Qual o número ideal de 'Personas' (clusters)? 2? 3? 5?
O 'Método do Cotovelo' é um teste para nos ajudar a decidir. Ele roda o algoritmo várias vezes (com K=1, K=2, K=3, ...) e mede o 'erro' (inércia) de cada um.
Nós plotamos esse erro. No começo, o erro cai *muito* (passar de 1 para 2 grupos ajuda). Mas em certo ponto, adicionar mais grupos não ajuda tanto. Esse ponto de 'retorno decrescente' é o 'cotovelo' (Elbow) e é um ótimo candidato para K."

🎓 **Definição Técnica:**
O **Elbow Method** é uma heurística para encontrar o número ótimo de clusters (K). Ele plota a **Inércia** (ou WCSS - Within-Cluster Sum of Squares) em função do número de clusters (K). A Inércia é a soma das distâncias ao quadrado de cada ponto até o centro do seu cluster. Procuramos o "cotovelo" (joelho) da curva, que é o ponto onde o WCSS começa a diminuir mais lentamente.

🍳 **Receita:**

In [ ]:
from sklearn.cluster import KMeans

inertia_list = []
K_range = range(1, 11) # Vamos testar de 1 a 10 clusters

for k in K_range:
    kmeans = KMeans(
        n_clusters=k,
        init='k-means++', # Método inteligente para espalhar os centros iniciais
        random_state=42,
        n_init=10 # Rodar 10x com centros diferentes e pegar o melhor
    )
    kmeans.fit(X_scaled)
    inertia_list.append(kmeans.inertia_) # 'inertia_' é o WCSS

# Plotar o gráfico do cotovelo
plt.figure(figsize=(8, 5))
plt.plot(K_range, inertia_list, 'bx-') # 'bx-' = linha azul com marcadores 'x'
plt.xlabel('Número de Clusters (K)')
plt.ylabel('Inércia (WCSS)')
plt.title('Método do Cotovelo (Elbow Method)')
plt.grid(True)
plt.show()

📊 **Resultado:**
"Olhe o gráfico. Onde a 'queda' para de ser íngreme e vira um 'cotovelo'? Geralmente é em K=3, K=4 ou K=5. Vamos supor que o cotovelo mais claro foi em **K=4**. Vamos usar K=4 para o nosso modelo final."

## Conceito 3: Treinando o Modelo Final (K-Means)

🧠 **Intuição:**
"Agora que escolhemos nosso K (ex: K=4) no Método do Cotovelo, rodamos o K-Means uma última vez.
O algoritmo faz o seguinte:
1. Joga 4 'ímãs' (centroides) aleatórios nos nossos dados.
2. Cada ponto de dado é 'puxado' para o ímã mais próximo.
3. O 'ímã' se move para o centro dos pontos que ele puxou.
4. Repete (2 e 3) até que os grupos não mudem mais.
No final, temos nossos 4 grupos (clusters) formados."

🎓 **Definição Técnica:**
Instanciamos e treinamos (com `.fit()`) o modelo K-Means usando o número de clusters (K) escolhido. O `n_init=10` é importante: ele roda o algoritmo 10 vezes com posições iniciais diferentes e escolhe a melhor (menor inércia), evitando cair em um "ótimo local" ruim. Após o `.fit()`, os rótulos dos clusters (0, 1, 2, 3) estarão disponíveis em `kmeans.labels_`.

🍳 **Receita:**

In [ ]:
# Vamos definir o K escolhido (ex: 4)
K_ESCOLHIDO = 4

# 1. Criar o modelo final
kmeans_final = KMeans(
    n_clusters=K_ESCOLHIDO,
    init='k-means++',
    random_state=42,
    n_init=10
)

# 2. Treinar e obter os rótulos (labels)
cluster_labels = kmeans_final.fit_predict(X_scaled)

# 3. Adicionar os rótulos de volta ao DataFrame *original*
#    Isso é vital para a interpretação!
df_original_com_clusters = df.copy()
df_original_com_clusters['Cluster'] = cluster_labels

print(f"Contagem de pessoas em cada Cluster (K={K_ESCOLHIDO}):")
print(df_original_com_clusters['Cluster'].value_counts().sort_index())
print("\nAmostra do DataFrame com Clusters:")
print(df_original_com_clusters.head())

📊 **Resultado:**
"Nosso DataFrame original agora tem uma nova coluna 'Cluster' (com valores de 0 a 3), dizendo a qual persona cada funcionário pertence."

## Conceito 4: Avaliação (Silhouette Score)

🧠 **Intuição:**
"O 'cotovelo' foi meio subjetivo. Será que K=4 foi *realmente* bom?
O 'Silhouette Score' é uma nota de -1 a +1 que nos diz o quão bons são os clusters.
* **+1:** Perfeito! Clusters são densos (pontos próximos) e muito bem separados (longe uns dos outros).
* **0:** Ruim. Clusters estão sobrepostos, os grupos não são claros.
* **-1:** Péssimo. Os pontos foram parar no cluster errado.
Queremos o valor mais próximo de +1 possível."

🎓 **Definição Técnica:**
O Coeficiente de Silhueta (`silhouette_score`) mede a qualidade do clustering. Ele calcula para cada ponto:
a) A distância média para os pontos no *mesmo* cluster (coesão).
b) A distância média para os pontos no cluster *mais próximo* (separação).
O score é $(b - a) / max(a, b)$. Um 'a' pequeno e um 'b' grande (score perto de 1) é o ideal.

🍳 **Receita:**

In [ ]:
from sklearn.metrics import silhouette_score

# Usamos os dados ESCALADOS e os rótulos (labels) que o K-Means gerou
score = silhouette_score(X_scaled, cluster_labels)
print(f"O Silhouette Score para K={K_ESCOLHIDO} foi de: {score:.4f}")

📊 **Resultado:**
"Um score de 0.5 a 0.7 é considerado um clustering forte. Um score de 0.2 a 0.4 é razoável/fraco. Abaixo de 0.2, os clusters não são bem definidos."

## Conceito 5: Interpretação (Criando as Personas)

🧠 **Intuição:**
"Esta é a parte mais importante. O modelo nos deu 'Cluster 0', 'Cluster 1', 'Cluster 2', 'Cluster 3'. E daí? Isso não diz nada para o RH.
Nós precisamos *humanizar* esses números. Vamos 'tirar a média' de cada cluster.
* Cluster 0: Média de Salário ALTA, média de Satisfação ALTA.
* Cluster 1: Média de Salário BAIXA, média de Satisfação BAIXA.
Agora sim! Podemos dar nomes a eles."

🎓 **Definição Técnica:**
A interpretação dos clusters (o passo mais crucial) é feita analisando os centroides de cada cluster. A forma mais fácil de fazer isso é usando `.groupby()` no DataFrame original (com os valores reais, *não* escalados) pela coluna 'Cluster' que criamos, e calculando a `.mean()` de cada feature.

🍳 **Receita:**

In [ ]:
# Agrupar pela coluna 'Cluster' e calcular a média das features que usamos
# (Incluímos 'pediu_demissao' para ver o risco de turnover de cada persona!)
features_para_analise = features_para_cluster + ['pediu_demissao']

# Usamos .drop() para remover colunas de texto que não podem ter média
df_personas = df_original_com_clusters.drop(columns=['departamento']).groupby('Cluster').mean()

# Adicionar o tamanho de cada cluster para contexto
df_personas['Tamanho (N)'] = df_original_com_clusters['Cluster'].value_counts()

print("--- Tabela de Personas (Médias por Cluster) ---")
# Formatando a tabela para melhor visualização
print(df_personas.style.format({
    'salario': 'R$ {:,.2f}',
    'satisfacao': '{:.2%}',
    'tempo_empresa': '{:.1f} anos',
    'avaliacao_performance': '{:.2%}',
    'pediu_demissao': '{:.2%}'
}).background_gradient(cmap='viridis', subset=['salario', 'satisfacao', 'avaliacao_performance']).background_gradient(cmap='coolwarm', subset=['pediu_demissao']))

📊 **Resultado (A Tabela de Personas):**
"Esta tabela é o resultado final do projeto. Agora podemos dar os nomes:
* **Cluster 0 (Ex):** Salário ALTO, Satisfação ALTA, Tempo de Empresa ALTO, Turnover BAIXO. -> **Persona: 'Veteranos Engajados'**.
* **Cluster 1 (Ex):** Salário BAIXO, Satisfação BAIXA, Tempo de Empresa BAIXO, Turnover ALTO. -> **Persona: 'Novatos em Risco'**.
* **Cluster 2 (Ex):** Salário MÉDIO, Satisfação MÉDIA, Perf. ALTA, Turnover MÉDIO. -> **Persona: 'Talentos Estagnados'**.
* ... e assim por diante."

## Conceito 6 (Bônus): Visualização dos Clusters (2D)

🧠 **Intuição:**
"Uma imagem vale mais que mil palavras. Vamos 'desenhar' os clusters. Escolhemos as duas features mais importantes (ex: 'salario' e 'satisfacao') e plotamos um gráfico de dispersão (scatterplot), colorindo cada ponto com a cor do seu cluster. Isso ajuda o RH a *ver* as 'bolhas' de personas que acabamos de identificar."

🎓 **Definição Técnica:**
A visualização de clusters é uma etapa qualitativa de validação. Como nossos dados têm muitas dimensões (ex: 5 ou 6 features), não podemos visualizá-los diretamente.
A técnica mais simples é plotar um gráfico de dispersão 2D usando as duas features mais interpretáveis (ou as mais importantes, segundo o `feature_importance_` de um modelo Random Forest).
Usamos o `hue='Cluster'` (do Seaborn) para mapear a coluna de rótulo do cluster (0, 1, 2, 3) a um esquema de cores, permitindo-nos ver a separação (ou sobreposição) dos grupos.

🍳 **Receita:**

In [ ]:
plt.figure(figsize=(10, 7))
sns.scatterplot(
    data=df_original_com_clusters,
    x='salario',
    y='satisfacao',
    hue='Cluster', # A mágica acontece aqui
    palette='viridis', # Esquema de cores
    s=100, # Tamanho dos pontos
    alpha=0.7 # Transparência
)
plt.title('Clusters de Funcionários (Personas)')
plt.xlabel('Salário')
plt.ylabel('Nível de Satisfação')
plt.legend(title='Persona (Cluster)')
plt.grid(linestyle='--', alpha=0.5)
plt.show()

📊 **Resultado:**
"O gráfico mostra visualmente as 'fronteiras' que o K-Means encontrou. Idealmente, veremos grupos de cores distintas (Ex: um 'bolsão' verde no canto inferior esquerdo, um 'bolsão' roxo no canto superior direito). Se as cores estiverem muito misturadas, isso reforça um Silhouette Score baixo e sugere que, pelo menos *nestas duas dimensões*, os grupos não são bem definidos."